In [ ]:
from openai import OpenAI
from openai import embeddings
from dotenv import load_dotenv

from IPython.display import Markdown, display


from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery, VectorQuery, VectorizedQuery, QueryType


load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]

# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
embedding_model = "text-embedding-3-small"
model = "gpt-5-mini"


True

In [93]:
# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [94]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

In [95]:
search_client = SearchClient(
    endpoint,
    index_name,
    credential=credential)



In [96]:

def generate_embeddings(text):
    client = OpenAI(
        api_key=api_key,
        organization=org,
        project=project
    )

    embedding = client.embeddings.create(
        model=embedding_model,
        input=text
    )
    embedding = embedding.data[0].embedding
    return embedding

In [97]:
# define a search function

def search_function(query_text, top_k=5, vector_only=False):
    vector = generate_embeddings(query_text)
    vector_query = VectorizedQuery(
        vector=vector,
        k_nearest_neighbors=top_k,
        fields="content_vector",
        exhaustive=True,
    )

    search_client = SearchClient(
        endpoint,
        index_name,
        credential=credential)

    search_text = None if vector_only else query_text
    results = search_client.search(
        search_text=search_text,
        vector_queries=[vector_query],
        select=["chunk_id", "file_name", "base_name", "content", "content_vector"],
        top=top_k,
        include_total_count=True,
        # query_type=QueryType.FULL,
        semantic_configuration_name="my-semantic-config",
    )
    return results

In [98]:
def search_display_result(query_text, vector_only=False):
    results  = search_function(query_text, vector_only=vector_only)
    for i, result in enumerate(results):
        display(Markdown(f"### Result {i+1}"))
        display(Markdown(f"**Chunk ID:** {result['chunk_id']}"))
        display(Markdown(f"**Score:** {result['@search.score']}"))
        display(Markdown(f"**Content:** {result['content']}"))
        display(Markdown("---"))

In [99]:
search_display_result("embeddings in AI", vector_only=True)

### Result 1

**Chunk ID:** Northwind_Health_Plus_Benefits_Details_0000

**Score:** 0.5680338

**Content:** # Contoso Electronics  
Northwind Health Plus Plan  
<figure>  
Contoso
Electronics  
</figure>  
<!-- PageBreak -->  
This document contains information generated using a language model (Azure OpenAI). The
information contained in this document is only for demonstration purposes and does not
reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or
warranties of any kind, express or implied, about the completeness, accuracy, reliability,
suitability or availability with respect to the information contained in this document.  
All rights reserved to Microsoft  
<!-- PageBreak -->

---

...

### Result 5

**Chunk ID:** PerksPlus_0000

**Score:** 0.5494856

**Content:** PerksPlus Health and Wellness
Reimbursement Program for
Contoso Electronics Employees  
<figure>  
Contoso
Electronics  
</figure>  
<!-- PageBreak -->  
This document contains information generated using a language model (Azure OpenAI). The information
contained in this document is only for demonstration purposes and does not reflect the opinions or
beliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied,
about the completeness, accuracy, reliability, suitability or availability with respect to the information
contained in this document.  
All rights reserved to Microsoft  
<!-- PageBreak -->

---

In [100]:
 # This prompt provides instructions to the model
 GROUNDED_PROMPT="""
 # PURPOSE:
 You are a friendly assistant that answer questions from company's HR policy and information documents.
 Answer the query using only the sources provided below in a friendly and concise bulleted manner.
 Answer ONLY with the facts listed in the list of sources below.
 If there isn't enough information below, say you don't know.
 Do not generate answers that don't use the sources below.

 # CITATION:
 Always provide citations inline from sources provided, and also include a list of sources as a foot note in bullet points format. example list of sources:
 Only include sources that were used to generate the answer. example list of sources:
 ### Sources:
 - SOURCE NO 1. filename1.pdf(chunk id number))
 - SOURCE NO 2. filename2.pdf(3)
 - SOURCE NO 3. filename3.pdf(5)
 Sources:\n{sources}

 Semantic Answer:\n{semantic_response}
 """

In [115]:

 # Query is the question being asked. It's sent to the search engine and the LLM.
# user_query="what the role of a project manager?"

user_query = input("Enter your question: ") or "am i covered for eye exams"

print(user_query)

What is the process for performance reviews?


In [116]:
#extract user search intent
SEARCH_INTENT_PROMPT = """
You are asked to read the user query, understand it, and then turn it into a search query.
The search query should be sent to the search engine to find the relevant information.
the search query should be similar to what a user would type into a search engine, and similar in meaning to the provided user query.
the user query is: '{user_query}'
return only the search query text. """

print(SEARCH_INTENT_PROMPT.format(user_query=user_query))


response = client.responses.create(
    model=model,
    input=[
        {"role": "system", "content": SEARCH_INTENT_PROMPT.format(user_query=user_query)},
    ]
)

search_query = response.output_text

print("search query intent: " , search_query)


You are asked to read the user query, understand it, and then turn it into a search query.
The search query should be sent to the search engine to find the relevant information.
the search query should be similar to what a user would type into a search engine, and similar in meaning to the provided user query.
the user query is: 'What is the process for performance reviews?'
return only the search query text. 
search query intent:  performance review process steps and best practices


In [117]:
 # Retrieve the selected fields from the search index related to the question.
results = search_function(search_query)


In [118]:
results.get_answers()

In [119]:


semantic_answers = results.get_answers()
semantic_response = ""
if semantic_answers:
    for answer in semantic_answers:
        if answer.highlights:
            semantic_response = answer.highlights
            # print(f"Semantic Answer: {answer.highlights}")
        else:
            semantic_response = answer.text
            # print(f"Semantic Answer: {answer.text}")
        print(f"Semantic Answer: {semantic_response}")
        print(f"Semantic Answer Score: {answer.score}\n")

In [120]:
list_of_sources = [f'\nSOURCE NO {index + 1}. {result["base_name"]}({result["chunk_id"]}): {result["content"]} )' for index, result in enumerate(results)]

In [121]:
print(list_of_sources)

["\nSOURCE NO 1. employee_handbook.md(employee_handbook_0004): ## Performance Reviews  \n### Performance Reviews at Contoso Electronics  \nAt Contoso Electronics, we strive to ensure our employees are getting the feedback they\nneed to continue growing and developing in their roles. We understand that performance\nreviews are a key part of this process and it is important to us that they are conducted in an\neffective and efficient manner.  \nPerformance reviews are conducted annually and are an important part of your career\ndevelopment. During the review, your supervisor will discuss your performance over the\npast year and provide feedback on areas for improvement. They will also provide you with\nan opportunity to discuss your goals and objectives for the upcoming year.  \nPerformance reviews are a two-way dialogue between managers and employees. We\nencourage all employees to be honest and open during the review process, as it is an\nimportant opportunity to discuss successes and 

In [122]:
joined_sources = "\n".join(list_of_sources)

In [123]:
print(joined_sources)


SOURCE NO 1. employee_handbook.md(employee_handbook_0004): ## Performance Reviews  
### Performance Reviews at Contoso Electronics  
At Contoso Electronics, we strive to ensure our employees are getting the feedback they
need to continue growing and developing in their roles. We understand that performance
reviews are a key part of this process and it is important to us that they are conducted in an
effective and efficient manner.  
Performance reviews are conducted annually and are an important part of your career
development. During the review, your supervisor will discuss your performance over the
past year and provide feedback on areas for improvement. They will also provide you with
an opportunity to discuss your goals and objectives for the upcoming year.  
Performance reviews are a two-way dialogue between managers and employees. We
encourage all employees to be honest and open during the review process, as it is an
important opportunity to discuss successes and challenges in t

In [124]:
GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response)
print(GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response))


# PURPOSE:
You are a friendly assistant that answer questions from company's HR policy and information documents.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.

# CITATION:
Always provide citations inline from sources provided, and also include a list of sources as a foot note in bullet points format. example list of sources:
Only include sources that were used to generate the answer. example list of sources:
### Sources:
- SOURCE NO 1. filename1.pdf(chunk id number))
- SOURCE NO 2. filename2.pdf(3)
- SOURCE NO 3. filename3.pdf(5)
Sources:

SOURCE NO 1. employee_handbook.md(employee_handbook_0004): ## Performance Reviews  
### Performance Reviews at Contoso Electronics  
At Contoso Electronics, we strive to ensure our employees are getting the feedback t

In [125]:
response = client.responses.create(
    model=model,
    input=[
        {"role": "system", "content": GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response)},
        {"role": "user", "content": user_query}
    ]
)

In [126]:
print(user_query)
print(search_query)

What is the process for performance reviews?
performance review process steps and best practices


In [127]:


display(Markdown(response.output_text))


- Performance reviews are conducted annually and are part of career development at Contoso Electronics. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- During the review, your supervisor discusses your performance over the past year and provides feedback on areas for improvement. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- Reviews provide an opportunity to discuss goals and objectives for the upcoming year. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- Reviews are a two-way dialogue; employees are encouraged to be honest and open about successes and challenges. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- The company aims to provide positive, constructive feedback to help employees develop and grow. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- Employees receive a written summary of the performance review (including a performance rating, feedback, and goals/objectives), and this summary is discussed during the review session. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))  
- Employees are encouraged to reach out to their managers with any questions or concerns and are supported throughout the process. (SOURCE NO 1. employee_handbook.md(employee_handbook_0004))

### Sources:
- SOURCE NO 1. employee_handbook.md(employee_handbook_0004))